# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [24]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [25]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [26]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [27]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning. (Recall the vector that captures gender in the previous "king - man + woman = queen" example you saw.)

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [28]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [29]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55561949, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [30]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [31]:
wv.most_similar('Texas')

[('Louisiana', 0.8194500207901001),
 ('Oregon', 0.816554844379425),
 ('Massachusetts', 0.8152135610580444),
 ('Pennsylvania', 0.8033424615859985),
 ('Ohio', 0.8031134605407715),
 ('Illinois', 0.7913594841957092),
 ('Georgia', 0.7913480997085571),
 ('Utah', 0.7898058891296387),
 ('Maryland', 0.7883143424987793),
 ('Arkansas', 0.7848700284957886)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [32]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.4304584562778473),
 ('much-vaunted', 0.40450674295425415),
 ('once-reliable', 0.3870781362056732),
 ('Admon', 0.37873366475105286),
 ('g.o.b', 0.3717184066772461),
 ('Post-DOMA', 0.3690004348754883),
 ('Trashbag', 0.367870569229126),
 ('Slammers', 0.3675604462623596),
 ('writer-producer', 0.36737027764320374),
 ('Hearken', 0.36582648754119873)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [33]:
wv['Texas']

array([ 1.6774799e+00,  3.2087058e-02,  4.2044592e-01,  1.0327395e+00,
        4.6180555e-01,  7.1135367e-04, -1.6445862e+00, -6.9063589e-02,
        6.8575662e-01,  1.7996891e+00, -6.4300776e-01,  1.5266684e+00,
       -7.5467110e-02, -2.1243565e+00, -6.9004399e-01,  4.5618355e-01,
       -1.4710326e+00,  4.2067221e-01, -5.6216188e-02, -1.3564701e+00,
        3.7593359e-01,  1.0160617e+00,  1.7343230e+00, -8.1978810e-01,
        4.9683809e-01,  6.2970459e-01, -1.9235966e+00, -2.1983192e+00,
       -1.7664222e+00, -2.3080006e-01, -3.6714585e+00, -6.7477643e-01,
       -1.9927238e+00,  2.2210047e-01, -8.4316522e-01,  2.0719618e-01,
       -2.2876515e+00,  1.3890737e+00, -2.1929680e-01,  1.4629984e-01,
       -3.1212610e-01, -4.3912983e-01, -5.4285491e-01,  1.0999306e+00,
        2.4441245e-01,  1.8755695e+00, -2.7972193e+00, -1.9984436e+00,
        5.2186775e-01, -3.4901259e+00,  2.0420706e+00, -3.2371705e+00,
       -6.1601746e-01,  1.1335585e+00, -2.2521710e+00, -7.7330500e-01,
      

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [34]:
wv.vectors

array([[ 0.46034503, -1.7557538 , -0.53463787, ..., -0.15841144,
         1.6623701 , -0.88544667],
       [-2.5060437 , -2.2613013 , -0.8929068 , ...,  0.38292626,
         2.2163832 , -0.5841748 ],
       [-1.2844613 , -3.2904148 , -0.8893804 , ...,  0.93879   ,
        -0.00898438, -0.85685784],
       ...,
       [ 0.1212905 ,  0.149973  , -0.08616409, ..., -0.07678723,
         0.02652597,  0.00740643],
       [ 0.03202739,  0.02117644, -0.00725071, ...,  0.0307615 ,
        -0.0616574 , -0.05134495],
       [ 0.11604502,  0.11797512, -0.06776185, ..., -0.13056105,
         0.03554837,  0.00943934]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [35]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6252496242523193),
 ('title', 0.6040729284286499),
 ('villain', 0.5846191048622131),
 ('princess', 0.5839098691940308),
 ('crown', 0.5775648355484009),
 ('brunette', 0.5740671157836914),
 ('revival', 0.5713955760002136),
 ('rendition', 0.5653492212295532),
 ('monster', 0.5622801780700684),
 ('character', 0.5522518754005432)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!

# Playing Around

In [36]:
wv.most_similar(positive=['beast', 'flower'])

[('coat', 0.7022534012794495),
 ('tiger', 0.6957404613494873),
 ('stone', 0.6796764135360718),
 ('jacket', 0.6753606796264648),
 ('yellow', 0.6680043935775757),
 ('sweater', 0.6616533994674683),
 ('silver', 0.659756600856781),
 ('ensemble', 0.6590238213539124),
 ('monster', 0.6585773229598999),
 ('sequence', 0.6558905839920044)]

In [37]:
wv.most_similar('silver')

[('flower', 0.6717434525489807),
 ('pink', 0.669437050819397),
 ('diamond', 0.6685705184936523),
 ('gold', 0.6661815643310547),
 ('sweater', 0.6636810898780823),
 ('pair', 0.6549100875854492),
 ('clutch', 0.6432092189788818),
 ('blue', 0.6423872113227844),
 ('stone', 0.6398930549621582),
 ('rock', 0.6366965770721436)]

In [38]:
wv.most_similar('blue')

[('yellow', 0.8273365497589111),
 ('pink', 0.8225677609443665),
 ('gray', 0.7999077439308167),
 ('orange', 0.7776132822036743),
 ('red', 0.7762811779975891),
 ('purple', 0.7591878175735474),
 ('coat', 0.7511742115020752),
 ('bright', 0.7443355321884155),
 ('metal', 0.7438474893569946),
 ('lipstick', 0.7416648864746094)]

In [39]:
wv.most_similar('Obama')

[('Trump', 0.6590725183486938),
 ('Maduro', 0.6585804224014282),
 ('Erdogan', 0.6543865203857422),
 ('Rouhani', 0.654132604598999),
 ('Macron', 0.6339868307113647),
 ('Bush', 0.6305160522460938),
 ('Hollande', 0.6285050511360168),
 ('Ashraf', 0.6203118562698364),
 ('Ghani', 0.6073459982872009),
 ('Netanyahu', 0.603117823600769)]

In [40]:
wv.most_similar(['Obama', 'Trump'])

[('Erdogan', 0.6952388286590576),
 ('Bush', 0.6650077700614929),
 ('Rouhani', 0.6583577990531921),
 ('Macron', 0.6422613859176636),
 ('Maduro', 0.6389861106872559),
 ('Drumpf', 0.628532886505127),
 ('Netanyahu', 0.6148338913917542),
 ('Putin', 0.588276207447052),
 ('Xi', 0.5865577459335327),
 ('Assad', 0.5860471725463867)]

In [41]:
wv.most_similar('Trump')

[('Drumpf', 0.7499136924743652),
 ('Obama', 0.6590725183486938),
 ('Trump-Themed', 0.6539327502250671),
 ('Glover', 0.6531128287315369),
 ('Trump/Katie', 0.6171742677688599),
 ('Wack', 0.6133873462677002),
 ('Sterling', 0.6130331754684448),
 ('Erdogan', 0.6120450496673584),
 ('Trump-Joe', 0.5975949168205261),
 ('Verrilli', 0.5884877443313599)]